# analisis

importamos Dataset de kaggle.

In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("arashnic/book-recommendation-dataset")

print("Path to dataset files:", path)

/Users/emilianosandoval/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/emilianosandoval/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /Users/emilianosandoval/.cache/kagglehub/datasets/arashnic/book-recommendation-dataset/versions/3


In [4]:
import pandas as pd
import os

dataset_path = "/Users/emilianosandoval/.cache/kagglehub/datasets/arashnic/book-recommendation-dataset/versions/3"

# List files in the dataset directory
files = os.listdir(dataset_path)
print("Files in dataset directory:", files)

Files in dataset directory: ['classicRec.png', 'Users.csv', 'recsys_taxonomy2.png', 'Ratings.csv', 'DeepRec.png', 'Books.csv']


In [5]:
# List all CSV files in the directory
csv_files = [file for file in os.listdir(dataset_path) if file.endswith(".csv")]

# Read each CSV file into a dictionary of DataFrames
dataframes = {file: pd.read_csv(os.path.join(dataset_path, file)) for file in csv_files}

# Print the keys (file names) to confirm
print("Loaded DataFrames:", dataframes.keys())

Loaded DataFrames: dict_keys(['Users.csv', 'Ratings.csv', 'Books.csv'])


/var/folders/x8/bcyrr1557g10v0ydrq5tr0gh0000gn/T/ipykernel_12391/3793803343.py:5: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  dataframes = {file: pd.read_csv(os.path.join(dataset_path, file)) for file in csv_files}


In [7]:
df_users = dataframes["Users.csv"]
df_ratings = dataframes["Ratings.csv"]
df_books = dataframes["Books.csv"]

In [ ]:
df_users.head()

,User-ID,Location,Age
0,1,"nyc, new york, usa",NaN
1,2,"stockton, california, usa",18.0
2,3,"moscow, yukon territory, russia",NaN
3,4,"porto, v.n.gaia, portugal",17.0
4,5,"farnborough, hants, united kingdom",NaN
